# Módulo 8: Usando MCP Servers Existentes
## Notebook 3: Agente con SQLite MCP

### 🎯 Objetivos
- Crear un agente LangGraph completo
- Integrar herramientas SQLite MCP
- Hacer consultas en lenguaje natural
- Experimentar con diferentes preguntas

### 📋 Pre-requisitos
- Haber completado Notebooks 1 y 2
- Servidor SQLite MCP corriendo en otra terminal
- Base de datos ejemplo.db creada


## Paso 1: Importar Dependencias y Configurar


In [ ]:
import asyncio
import os
from pathlib import Path
from dotenv import load_dotenv

# Cargar API key
root_dir = Path().absolute().parent
env_path = root_dir / '.env'
load_dotenv(env_path)

# Verificar API key
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("GOOGLE_API_KEY no encontrada en .env")

print(f"✅ API Key configurada (longitud: {len(api_key)} caracteres)")


## Paso 2: Crear el Agente


In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, MessagesState
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_core.messages import SystemMessage, HumanMessage

async def crear_agente():
    """Crea un agente que usa SQLite MCP"""
    
    # 1. Conectar con SQLite MCP
    print("🔌 Conectando con SQLite MCP...")
    client = MultiServerMCPClient({
        "sqlite": {
            "transport": "streamable_http",
            "url": "http://localhost:3000/mcp"
        }
    })
    
    tools = await client.get_tools()
    print(f"✅ {len(tools)} herramientas disponibles")
    
    # 2. Configurar LLM
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
    llm_with_tools = llm.bind_tools(tools)
    
    # 3. System message
    system_message = SystemMessage(
        content="""Eres un asistente experto en análisis de datos.
        
Tienes acceso a una base de datos con tablas de clientes, productos y ventas.
Usa las herramientas disponibles para responder preguntas del usuario.
Proporciona respuestas claras y bien estructuradas."""
    )
    
    # 4. Definir nodo asistente
    def assistant_node(state: MessagesState):
        messages = [system_message] + state["messages"]
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # 5. Construir grafo
    builder = StateGraph(MessagesState)
    builder.add_node("assistant", assistant_node)
    builder.add_node("tools", ToolNode(tools))
    
    builder.add_edge(START, "assistant")
    builder.add_conditional_edges("assistant", tools_condition)
    builder.add_edge("tools", "assistant")
    
    print("🤖 Agente creado exitosamente!")
    return builder.compile()

# Crear agente
graph = await crear_agente()


## Paso 3: Probar el Agente

Ahora puedes hacer preguntas en lenguaje natural y el agente usará las herramientas del servidor SQLite MCP para responder.


In [ ]:
# Ejemplo 1: Contar clientes
pregunta = "¿Cuántos clientes hay en la base de datos?"
print(f"🤔 Pregunta: {pregunta}\\n")

result = await graph.ainvoke({
    "messages": [HumanMessage(content=pregunta)]
})

respuesta = result["messages"][-1].content
print(f"🤖 Respuesta:\\n{respuesta}")


In [ ]:
# Ejemplo 2: Productos más caros
pregunta = "Muéstrame los 3 productos más caros"
print(f"🤔 Pregunta: {pregunta}\\n")

result = await graph.ainvoke({
    "messages": [HumanMessage(content=pregunta)]
})

respuesta = result["messages"][-1].content
print(f"🤖 Respuesta:\\n{respuesta}")


## 🎯 Ejercicios

Prueba hacer estas preguntas al agente:

1. "¿Qué clientes son de Madrid?"
2. "¿Cuál fue la venta con mayor monto?"
3. "¿Cuántos productos hay en la categoría de Electrónica?"
4. "Muéstrame las ventas del último mes"

Crea nuevas celdas abajo y experimenta con tus propias preguntas!


## ✅ ¡Felicitaciones!

Has creado exitosamente un agente LangGraph que usa un servidor MCP existente.

### ⏭️ Próximo Paso
Continúa con `04_multi_servidor.ipynb` para aprender a usar múltiples servidores MCP simultáneamente.
